In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from ml_toolkit.data_handler import DataHandler
from ml_toolkit.feature_engineer import FeatureEngineer
from ml_toolkit.model_builder import ModelBuilder

In [2]:
start_date="2020-1-1 00:00"
end_date="2024-07-29 21:00"

# Load the data
df_GBP_USD_H4 = DataHandler.load_data('../fin_data/GBP_USD_H4.pkl', start_date, end_date)
df_EUR_USD_H4 = DataHandler.load_data('../fin_data/EUR_USD_H4.pkl', start_date, end_date)
df_GBP_JPY_H4 = DataHandler.load_data('../fin_data/GBP_JPY_H4.pkl', start_date, end_date)
df_AUD_USD_H4 = DataHandler.load_data('../fin_data/AUD_USD_H4.pkl', start_date, end_date)
df_EUR_CHF_H4 = DataHandler.load_data('../fin_data/EUR_CHF_H4.pkl', start_date, end_date)
df_EUR_GBP_H4 = DataHandler.load_data('../fin_data/EUR_GBP_H4.pkl', start_date, end_date)
df_XAU_USD_H4 = DataHandler.load_data('../fin_data/XAU_USD_H4.pkl', start_date, end_date)

Loading raw data from pickle file: '../fin_data/GBP_USD_H4.pkl'...
Filtering data from 2020-01-01 to 2024-07-29.
Data successfully loaded and filtered. Resulting shape: (7130, 6)
Time range in cleaned data: 2020-01-01 22:00:00 to 2024-07-29 21:00:00
Loading raw data from pickle file: '../fin_data/EUR_USD_H4.pkl'...
Filtering data from 2020-01-01 to 2024-07-29.
Data successfully loaded and filtered. Resulting shape: (7130, 6)
Time range in cleaned data: 2020-01-01 22:00:00 to 2024-07-29 21:00:00
Loading raw data from pickle file: '../fin_data/GBP_JPY_H4.pkl'...
Filtering data from 2020-01-01 to 2024-07-29.
Data successfully loaded and filtered. Resulting shape: (7130, 6)
Time range in cleaned data: 2020-01-01 22:00:00 to 2024-07-29 21:00:00
Loading raw data from pickle file: '../fin_data/AUD_USD_H4.pkl'...
Filtering data from 2020-01-01 to 2024-07-29.
Data successfully loaded and filtered. Resulting shape: (7130, 6)
Time range in cleaned data: 2020-01-01 22:00:00 to 2024-07-29 21:00:00


In [ ]:
# Define features
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'MACD', '%K', 'CO', 'CU', 'MA50']
            
# Feature Engineering
df, time, X, y = FeatureEngineer.add_features(df=df_GBP_USD_H4, features=features)
df.drop(['Signal'], axis=1, inplace=True)
df['Signal'] = FeatureEngineer.find_peaks_and_valleys(df[['Time', 'Close']], distance=5, prominence=0.005)
df['GBP_USD'] = df_GBP_USD_H4['Close']
df['EUR_USD'] = df_EUR_USD_H4['Close']
df['GBP_JPY'] = df_GBP_JPY_H4['Close']
df['AUD_USD'] = df_AUD_USD_H4['Close']
df['EUR_CHF'] = df_EUR_CHF_H4['Close']
df['EUR_GBP'] = df_EUR_GBP_H4['Close']
df['XAU_USD'] = df_XAU_USD_H4['Close']

# Specify features and target in financial dataset
features = ['RSI', 'MACD', '%K', 'GBP_USD', 'EUR_USD', 'GBP_JPY', 'AUD_USD', 'EUR_CHF', 'EUR_GBP', 'XAU_USD']
target = ['Signal']

# Preprocessing steps (typically done outside ModelBuilder, perhaps in DataHandler)
df = df.dropna()
X = df[features]
y = df[target]
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# -------------------------------------------------
# --- Build and Train a Neural Network Model  -----
# -------------------------------------------------
print("--- Training Neural Network ---")
nn_model = ModelBuilder.build_and_train_regression_model(
    X_train=X_train,
    y_train=y_train,
    features=features,
    target=target,
    model_type='nn',
    nn_epochs=50,
    nn_batch_size=32
)
if nn_model:
    print("Neural Network Model built and trained successfully.")
    # Pass nn_model to a ModelEvaluator for prediction and evaluation

# -------------------------------------------------
# --- Build and Train a Random Forest Regressor ---
# -------------------------------------------------
print("\n--- Training Random Forest Regressor ---")
rf_model = ModelBuilder.build_and_train_regression_model(
    X_train=X_train,
    y_train=y_train,
    features=features,
    target=target,
    model_type='rf'
)
if rf_model:
    print("Random Forest Model built and trained successfully.")
    # Pass rf_model to a ModelEvaluator for prediction and evaluation
